In [1]:
from shelve import open as open_shelve
from matplotlib import pyplot as plt
%matplotlib qt
from kitt_net import FeedForwardNet
from sklearn.metrics import confusion_matrix, accuracy_score
from numpy import newaxis, zeros

In [2]:
dataset = open_shelve('../examples/karnin/dataset_karnin.ds')

In [3]:
how = {1: 'b.', -1: 'r.'}
for x, y in zip(dataset['x'], dataset['y']):
    plt.plot(x[0], x[1], how[y])
plt.grid()
plt.show()

In [3]:
net = FeedForwardNet(hidden=[5], tf_name='Tanh')

In [4]:
n_epoch = 100
learning_rate = 0.05
net.fit(x=dataset['x'], y=dataset['y'], x_val=dataset['x_val'], y_val=dataset['y_val'],
       learning_rate=learning_rate, n_epoch=n_epoch, batch_size=16)


--------------------------------------------------------------------
-- Network initialized.
	% problem dimension: 2
	% number of classes: 2
	% class labels: [-1.0, 1.0]
	% net structure: [2, 5, 2]
	% net transfer function: Tanh

--------------------------------------------------------------------
-- Learning has started...
	% problem dimension: 2
	% number of training samples: 1600
	% number of validation samples: 200
	% learning rate: 0.05
	% mini-batch size: 16
	% maximum number of epochs (t.c.): 100
	% maximum number of stable epochs (t.c.): inf
	% required accuracy (t.c.): inf
	% required error (t.c.): -inf


epoch   on training data      on validation data       epoch time          
-------------------------------------------------------------------
 1	  0.91/0.1240		0.91/0.1154		0.0904 s
 2	  0.93/0.0947		0.90/0.1246		0.0652 s
 3	  0.97/0.0462		0.95/0.0442		0.0682 s
 4	  0.97/0.0396		0.97/0.0427		0.0514 s
 5	  0.93/0.1083		0.94/0.0863		0.0754 s
 6	  0.97/0.0398		0.96/0.0316		0.

In [9]:
print net.dw_container[0]
#print net.w[0]
#print net.b[0]
#print '----------------'
#print net.w[1]
#print net.b[1]
#print 'xxxxxxxxxxxxxxxxxxxxxxxxxx'
# Weight change
#print abs(net.w_init[0]-net.w[0])
#print abs(net.b_init[0]-net.b[0])
#print net.dw_container[0]
#print '----------------'
#print abs(net.w_init[1]-net.w[1])
#print abs(net.b_init[1]-net.b[1])
#print net.dw_container[1]

[array([[-22.23671618,  45.31671828],
       [ 11.76193429,  17.4629401 ],
       [  1.79397746,   0.52164295],
       [ 43.19954778, -11.6725141 ],
       [  1.34899322,   4.37340996]]), array([[  3.76472104,  37.28671087],
       [  5.3004449 ,   6.38130328],
       [  0.32836264,   0.26993167],
       [ 21.761683  ,  -2.71447376],
       [  1.41858753,   1.62876179]]), array([[ -4.10401657,  14.20923252],
       [  1.22793817,   2.90789313],
       [  0.13552236,   0.33105945],
       [ 12.93374873,  -2.93683364],
       [  0.46521826,   1.40290723]]), array([[  1.39044291,  10.69535237],
       [  2.73824628,   1.60838766],
       [  0.12222959,   0.16075265],
       [ 10.69796309,   1.16834127],
       [  0.93724473,   0.7991326 ]]), array([[ 0.79306526,  7.16951352],
       [ 2.2262173 ,  1.3040192 ],
       [ 0.09215796,  0.30198073],
       [ 7.98319489,  1.57105261],
       [ 0.84022816,  0.98062892]]), array([[ -5.56159488,  17.01129066],
       [  1.36533759,   2.01812906],


In [20]:
# Karnin sensitivities
s = [zeros(shape=w_i.shape) for w_i in net.w]
for ep_i in range(n_epoch):
    for l_i in range(len(s)):
        s[l_i] += net.dw_container[l_i][ep_i]**2 * (net.w[l_i]/(learning_rate*(net.w[l_i]-net.w_init[l_i])))

print s

[array([[ 4144.69669904,  8142.66407088,  -181.22122115,   192.60513786],
       [ 4355.77392173,  2038.69191119,  -123.66912155,    17.72412337],
       [  916.08618035,   272.07482858,   332.18451858,   692.9974425 ]]), array([[ 11531.50754774,  10746.17695456,   2411.70462884],
       [  9204.65668354,   6555.84540512,   2473.74145156]])]


In [5]:
net.prune(req_acc=0.98, n_epoch=50, measure='karnin')


--------------------------------------------------------------------
-- Pruning has started...
	% net initial structure: [2, 5, 2]
	% net initial number of synapses (w, b): (20, 7)
	% min required accuracy (s.c.): 0.98
	% max required error (s.c.): 0.0
	% maximum number of re-training epochs (gu.c.): 50
	% number of stable iterations (gu.c.): 10


step    trying to cut     structure           left synapses (w/b)    retrained      next level     step time      
--------------------------------------------------------------------------------------------------------------
0       0                 [2, 5, 2]           (20, 7)                None           75             None  
1       15                [2, 1, 2]           (2, 3)                 no             50             5.1004 s
2       10                [2, 2, 2]           (6, 4)                 yes            50             5.4832 s
3       3                 [1, 1, 2]           (3, 3)                 no             35             5.

In [10]:
print net.w_is

[array([[ 1.,  0.],
       [ 0.,  1.]]), array([[ 1.,  1.],
       [ 0.,  1.]])]


In [11]:
labels = [label for label in net.labels if label in dataset['y_val']]
y_pred = [net.predict(x)[0][0] for x, y in net.v_data]
print 'Acc:', accuracy_score(y_true=dataset['y_val'], y_pred=y_pred)
cm = confusion_matrix(y_true=dataset['y_val'], y_pred=y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, newaxis]
plt.imshow(cm, aspect='auto', interpolation='none', vmin=0, vmax=1)
plt.xticks(range(len(labels)), labels)
plt.yticks(range(len(labels)), labels)
plt.grid()
plt.colorbar()
plt.show()
print cm
for i in range(cm.shape[0]):
    print i, net.labels[i], sorted([(net.labels[j], round(cm[i, j], 2)) for j in range(cm.shape[1])], key=lambda x:x[1], reverse=True)[:5]


Acc: 0.98
[[ 1.    0.  ]
 [ 0.04  0.96]]
0 -1.0 [(-1.0, 1.0), (1.0, 0.0)]
1 1.0 [(1.0, 0.96), (-1.0, 0.04)]


In [13]:
print net.b_init

[array([[ 0.60948547],
       [-1.61106314]]), array([[ 0.66942606],
       [ 0.08837358]])]
